# Helium atom

Once we have a solver for the Hydrogen Atom (1 atom), the next level off difficulty is the Helium atom (2 atoms). Here we'll show how
to make and SCF for the Helium atom within the framework of Hartree&ndash;Fock.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np

def laplace_operator(D, f_tree):
    return D(D(f_tree, 0), 0) + D(D(f_tree, 1), 1) + D(D(f_tree, 2), 2)

def calculate_energy(phi_tree, V_tree):
    mra = phi_tree.MRA()
    d_oper = vp.ABGVDerivative(mra, 0.5, 0.5)
    return -0.5*vp.dot(laplace_operator(d_oper, phi_tree), phi_tree) + vp.dot(phi_tree, V_tree*phi_tree)

def couloumb_potential(prec, phi_tree):
    mra = phi_tree.MRA()
    P = vp.PoissonOperator(mra, prec)
    return P(4.0*np.pi*phi_tree*phi_tree)

# Analytic nuclear potential: f_nuc(r) = Z/|r|
def f_nuc(r):
    Z = 2.0                 # Nuclear charge
    R = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
    return -Z / R

# Analytic guess for wavefunction: f_phi(r) = exp(-r^2)
def f_phi(r):
    R2 = r[0]**2 + r[1]**2 + r[2]**2
    return np.exp(-R2)

# Set target precision
precision = 1.0e-5

# Define MRA basis and computational domain
k = 5                       # Polynomial order of MRA basis
world = [-20, 20]           # Computational domain [-L,L]^3 (a.u.)
MRA = vp.MultiResolutionAnalysis(order=k, box=world)

# Define projector onto the MRA basis
P_mra = vp.ScalingProjector(mra=MRA, prec=precision)

# Initialize the calculation
phi_n = P_mra(f_phi)           # Project analytic guess for wavefunction onto MRA
phi_n.normalize()              # Normalize the wavefunction guess
V_n = P_mra(f_nuc)               # Project analytic nuclear potential onto MRA
J = couloumb_potential(precision, phi_n)
E_n = calculate_energy(phi_n, V_n + J)

In [ ]:
# Loop parameters
iteration = 0                  # Iteration counter
max_iter = 30                  # Maximum iterations 
thrs = precision # -1           # Convergence requirement. Set to -1 if you wish to limit using max_iter
update = 1.0                   # Initialize error measure (norm of wavefunction update)
# Minimization loop
while (update > thrs):
    if iteration > max_iter-1:
        break
    # Build Helmholtz operator from current energy
    mu = np.sqrt(-2*E_n)
    G = vp.HelmholtzOperator(mra=MRA, exp=mu, prec=precision)
    
    # Apply Helmholtz operator
    phi_np1 = -2*G((V_n + J)*phi_n)
    
    # Compute wavefunction and energy update
    d_phi_n = phi_np1 - phi_n
    update = d_phi_n.norm()

    # Prepare for next iteration
    phi_n = phi_np1
    phi_n.normalize()
    J = couloumb_potential(precision, phi_n)
    E_n = calculate_energy(phi_n, V_n + J)

    # Collect output
    print(iteration, " |  E:", E_n, " |  d_phi:", update)
    iteration += 1
E_tot = 2.0*E_n - vp.dot(phi_n*phi_n, couloumb_potential(precision, phi_n))